In [3]:
import scipy 
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.widgets import Slider, Button
import seaborn as sns
import numpy as np 
import random
import pandas as pd 
from ipywidgets import *
from scipy.stats import norm
# global variable, reproduct the same simualtion resutls if set true. 
set_random_seed = False
t =  np.linspace(90,0,91)/365
Real_vol = 0.5
Growth_rate =0
LTV0 = 0.8
bs_vol = 0.5
y_price = 50000
Collateral = 10000000
pool_init_x = 5000000
pool_init_y = 100
pool_init_wx = 0.5
fee_rate = 0 
rebate = 0 

In [2]:
t =  np.linspace(90,0,91)/365
delta_t = t[:-1]-t[1:]
Growth_rate=-2
rw = np.random.random(size = len(t)-1)
delta_price = delta_t * (Growth_rate )
delta_price.cumsum()

array([-0.00547945, -0.0109589 , -0.01643836, -0.02191781, -0.02739726,
       -0.03287671, -0.03835616, -0.04383562, -0.04931507, -0.05479452,
       -0.06027397, -0.06575342, -0.07123288, -0.07671233, -0.08219178,
       -0.08767123, -0.09315068, -0.09863014, -0.10410959, -0.10958904,
       -0.11506849, -0.12054795, -0.1260274 , -0.13150685, -0.1369863 ,
       -0.14246575, -0.14794521, -0.15342466, -0.15890411, -0.16438356,
       -0.16986301, -0.17534247, -0.18082192, -0.18630137, -0.19178082,
       -0.19726027, -0.20273973, -0.20821918, -0.21369863, -0.21917808,
       -0.22465753, -0.23013699, -0.23561644, -0.24109589, -0.24657534,
       -0.25205479, -0.25753425, -0.2630137 , -0.26849315, -0.2739726 ,
       -0.27945205, -0.28493151, -0.29041096, -0.29589041, -0.30136986,
       -0.30684932, -0.31232877, -0.31780822, -0.32328767, -0.32876712,
       -0.33424658, -0.33972603, -0.34520548, -0.35068493, -0.35616438,
       -0.36164384, -0.36712329, -0.37260274, -0.37808219, -0.38

In [15]:
class weights_generator: 
    """
        Purpose: generate weights based on different strategies 
                1.  Black-shoals option pricing model and observed price. 
                    A delta replicating strategy of a call option on Token / Collateral. 
                2.  Exp. smooth of (1):  0.95 * w(t-1) +  0.05 * w(t)
                3.  Moving average of N
        
        Assumption: 
            1. Rebalance following the above strategies.  
            2. Choice 1: Price percentage change (approximately equals to log(p(t+1)/p(t))) follows a linear growth treand 
               (slope = growth rate) plus a random walk Brownie motion. 
               Choice 2: Real historical btc price 
               
        Input: (attributes) 
            - t: term of the loan 
            - y_price0: [float/int] initial price y token (e.g. 1 btc = 50000 btc) 
            - bs_vol: black shoral option volatilty 
            - Growth_rate: linear growth slope of y price (anulized returns) 
            - Real_vol: growth Brownie motion volatility 
            - price_source: [numpy array] if acctual price, use [0] if generate price 
            
            Additional input:  *args 
                options: 
                ('BS') - Black-shoals weight 
                ('MA', ma_window)  - ma_window: weights moving average window.   
                                                Pad 0.5 if t < ma_window 
                ('SM', sm_wt) - sm_wt: expotential smothing weight: e.g. 0.95 
                ('50/50') - Uniswap type of pool, serving as a reference 
       
        Methods: 
            - BS_weights(*args) 
                - "BS"
                - 'MA', ma_window 
                - 'SM', wt 
                - '50/50' reference
    
        Output: 
            - A dataframe with columns: ['t', 'y_price', 'weights_ytoken', 'imp_loss_rebalance', 'imp_loss_price'] 
    
    """
    def __init__(self, t, y_price0, y_price_strike, bs_vol, Growth_rate, Real_vol, price_source, pool_init_wy):
        self.t = t
        self.y_price0 = y_price0
        self.bs_vol = bs_vol 
        self.Growth_rate = Growth_rate
        self.Real_vol = Real_vol 
        self.price_source = price_source
        self.y_price_strike = y_price_strike
        self.pool_init_wy = pool_init_wy

    def BS_weights(self, *args): 
       # set a global variable, to replicate all simualtion results if set true 
        if set_random_seed == True:
            np.random.seed(101)
       # loan life duation between t(i) - t(i+1)
        delta_t = self.t[:-1]-self.t[1:]
        if self.price_source[0] == 0:
            # token price percent change following a linear growth curve plus a random walk with volatility = Real_vol 
            rw = np.random.random(size = len(self.t)-1)
            delta_price = delta_t * (self.Growth_rate - 0.5*self.Real_vol**2) +  self.Real_vol *np.sqrt(delta_t)*norm.ppf(rw)
            # calcualte price with exponetial of cumulative sum of log price differences.  
            price_logdiff= np.hstack([0, delta_price]).cumsum()
            price = self.y_price0*np.exp(price_logdiff)
            if self.y_price0 != self.y_price_strike: 
                price_logdiff = price_logdiff + np.log(self.y_price0) - np.log(self.y_price_strike)
        elif self.price_source[0]>0:
            price = self.price_source
            price = np.append(price, price[-1])
            price_logdiff = np.log(price/price[0])
            if self.y_price0 != self.y_price_strike: 
                price_logdiff = price_logdiff + np.log(self.y_price0) - np.log(self.y_price_strike)

        # reblance weights by black scholes option equestion. 
        w_y = norm.cdf((price_logdiff[:-1] + self.t[:-1]*(0.5*self.bs_vol**2))/self.bs_vol/np.sqrt(self.t[:-1]))
        # set a boundary [1e-8, 1-1e-8] to avoid 0 as denominator 
        w_y[w_y< 1e-8] = 1e-8   
        w_y[w_y> (1- 1e-8)] = 1- 1e-8
        # no rebalance at the end of loan, so weights remains.  
        weights_ytoken = np.append(w_y, w_y[-1])
        args = list(np.array(args).flat)
        #print(args)
        if args[0] == 'BS':
            _wt = weights_ytoken
        elif args[0]== 'MA':
            _wt = self.moving_avg(weights_ytoken, args[1])
        elif args[0] == 'SM':
            _wt = self.sm(weights_ytoken, float(args[1]))
        elif args[0]== '50/50':
            _wt = np.array([0.5]*len(weights_ytoken))
        elif args[0]== 'SM-cap':
            _wt = self.sm_cap(weights_ytoken, float(args[1]), float(args[2]))
        else:
            print('no such smooth method')
        _wt_pad = np.append(self.pool_init_wy, _wt)   
        # Impermanent Loss (IL) due to weights change  
        imp_loss_wt = (_wt_pad[:-1]/_wt_pad[1:])**_wt_pad[1:]*((1-_wt_pad[:-1])/(1-_wt_pad[1:]))**(1-_wt_pad[1:])-1
        # IL due to price change 
        _a = (price[1:]/price[:-1])**_wt[:-1]
        _b = (price[1:]/price[:-1])*_wt[:-1] + 1*(1-_wt[:-1])
        imp_loss_price = np.append(np.divide(_a, _b, out=np.zeros_like(_a), where=_b!=0), 1) -1
   
        # formate the output file as a dataframe for downstream analysis 
        _temp= np.column_stack((t, price, _wt, imp_loss_wt, imp_loss_price,np.append(w_y, w_y[-1])))
        out_vec = pd.DataFrame(_temp, columns = ['t', 'y_price', 'weights_ytoken', 'imp_loss_rebalance', 'imp_loss_price', 'delta_wt'])
        return out_vec 

    def moving_avg(self, wt, ma_window):
        ma_window = int(ma_window)
        assert isinstance(ma_window, int), 'Moving average window must be integer!' 
        # pad the weights by initial weights 
        _w_padding = np.append(np.array([wt[0]]*(ma_window-1)), wt)
        _wt_ma = np.convolve(_w_padding, np.ones(ma_window)/ma_window, mode = 'valid')
        return _wt_ma

    def sm(self, wt, exp_w): 
        assert 0<=exp_w<=1, 'smoothing weight must be between 0 and 1!'
        _wt_sm = []
        _newwt = self.pool_init_wy
        for i in range(len(wt)):
            _newwt = _newwt*exp_w + wt[i]*(1-exp_w)
            _wt_sm.append(_newwt) 
        return np.array(_wt_sm)

    def sm_cap(self, wt, exp_w, th): 
        assert 0<=exp_w<=1, 'smoothing weight must be between 0 and 1!'
        _wt_sm = []
        _newwt = self.pool_init_wy
        for i in range(len(wt)):
            _temp = _newwt
            _newwt = _newwt*exp_w + wt[i]*(1-exp_w)
            if np.abs(_newwt - _temp) < th:
                _newwt = _temp
            _wt_sm.append(_newwt) 
        return np.array(_wt_sm)


In [16]:

class PoolEngine:
    """
       pool engine class
        Reflect real senario when weight changes makes arbitrage oppurtunites and pool value loss 
        
        Attribute: 
            - x: token 1 balance
            - y: token 2 balance
            - w_x: weight of x 
            - collateral value 
            - collateral with rebate 
            - fee
            - rebate 
            
        Method:
            - sp(): spot price of one y token in terms of x token
            - in-given-out(inTtype, amount): size of in given out amount 
            - out-given-in(inTtype, amount): size of our given in amount 
            - in_given_price(inTtype, p): size of in to bring spot price to p the price of out token as function of in token
            - step(new_wx, oracle_p, verbose = 0): attributes updated if weights or price changes
    """

    def __init__(self, x, y, w_x):
        assert 0 < w_x < 1, "weights must be in (0,1) !"
        self.x = x
        self.y = y
        self.w_x = w_x
        self.value = x + y * self.sp()

    def step(self, new_wx, oracle_p, verbose=0):
        """
            attributes updated if weights or price changes
            input: 
                - new_wx: weights of x 
                - oracle_p: oracle (market) price of y given x 
            
            attributes updated:
                - self.x 
                - self.y
                - self.w_x
                - self.value
            output:
                - delta_x 
                - delta_y
                - value change as unit of x 
                - _v impermanent loss 
                - impermanent loss percent
                - pool value change 
        """
        assert 0 < new_wx < 1, "weights must be in (0,1) !"
        self.w_x = new_wx
        spot_price = self.sp()
        value = self.value
        delta_x = self.in_given_price('x', oracle_p)
        delta_y = self.in_given_price('y', 1/oracle_p)
        self.x += delta_x
        self.y += delta_y
        _v = delta_x + delta_y*oracle_p
        self.value = self.x + self.y * oracle_p
        value_change = _v/value
        if verbose == 1:
            print('delta_x={}, and delta_y={:06.2f}, price ={:06.2f}, imp_loss={} '.format(delta_x, delta_y, oracle_p, value_change))
        return np.array([delta_x, delta_y, value_change, _v, (self.value-_v-value)/value, self.value-_v-value])

    def sp(self):
        """
            spotprice (implied price)
        """

        w_y = 1 - self.w_x
        p = self.x * w_y / (self.y * self.w_x)
        return p

    def in_given_out(self, inTtype, amount):
        if inTtype == 'x':
            assert 0 <= amount < self.y, 'out amount must be greater than 0 and less than total'
            ratio = (1 - self.w_x) / self.w_x
            Ain = self.x * ((self.y / (self.y - amount)) ** (ratio) - 1)
        else:
            assert 0 <= amount < self.x, 'out amount must be greater than 0 and less than total'
            ratio = self.w_x / (1 - self.w_x)
            Ain = self.y * ((self.x / (self.x - amount)) ** (ratio) - 1)
        return Ain

    def out_given_in(self, inTtype, amount):
        assert amount >= 0, 'amount must be greater than 0'
        if inTtype == 'x':
            ratio = self.w_x / (1 - self.w_x)
            Aout = self.y * (1 - (self.x / (self.x + amount)) ** ratio)
        else:
            ratio = (1 - self.w_x) / self.w_x
            Aout = self.x * (1 - (self.y / (self.y + amount)) ** ratio)
        return Aout

    def in_given_price(self, inTtype, price):
        assert price > 0, 'price must be greater than 0'
        if inTtype == 'x':
            sp_price = self.sp()
            Ain = self.x * ((price / sp_price) ** (1 - self.w_x) - 1)
        else:
            sp_price = 1 / self.sp()
            Ain = self.y * ((price / sp_price) ** self.w_x - 1)
        return Ain
    

In [17]:
def gen_data_vec(w_btc, price, pool_init_x, pool_init_y, imp_loss_rebalance, V0, g0, rebate=0):
    
    """
        Calculate the key pool metrics using vectorization (to speed up the process, equivalent to stepwise intertion.)
        input : output from weights_generator.BS_weights(args) 
        - weight: [numpy array] btc weight 
        - price: [numpy array] btc price 
        - pool_init_x [float/int] 
        - pool_init-y [float/int]
        - imp_loss_rebalance [numpy array]  
        - V0: initial invariance function
        - g0: initial pool value  
        - rebate: percent of impermanent loss due to rebalance  
        
        generate: Dataframe with the following columns  
        - x: usd size  
        - y: btc size 
        - V: invariance function 
        - G: portfolio value
        - rebate: invariant function changes factor after rebate 
        - imp_empirical: value loss delta_x + delta_y * p 
        - delta_x  
        - delta_y 
    """
    w_y = w_btc.values
    w_x = 1-w_y
    p = price.values 
    p_pad = np.append(p, p[-1])
    slip = imp_loss_rebalance.values
    # Update invariance function V by factor 
    _factor = (w_y[:-1]/(w_x[:-1]*p_pad[1:-1]))**(w_x[:-1]*w_y[1:]) * (w_x[:-1]*p_pad[1:-1]/(w_y[:-1]))**(w_y[:-1]*w_x[1:])
    #add rebate the inv-function update factor
    _rebate = (1+slip*(1-rebate))/(1+slip)
    _factor = _factor*_rebate[1:]
    V = np.hstack([V0, V0*_factor.cumprod()])
    y = V*((w_y/(w_x*p_pad[1:]))**w_x)
    x = V*((w_x*p_pad[1:])/w_y)**w_y
    g = x + y*p_pad[1:] 
    
    # calculate empirical impermanent loss: delta_x + delta_y * P 
    _x = np.diff(np.append(pool_init_x, x))
    _y = np.diff(np.append(pool_init_y, y))
    imp_empirical = (_x + _y*p_pad[1:])/np.append(g0,g[:-1])

    df = pd.DataFrame(np.column_stack((x,y,V,g,_rebate,imp_empirical, _x, _y)), columns = 
                      ['x_locked', 'y_locked','V','coll_with_rebate', 'rebate', 'imp_empirical','delta_x', 'delta_y'])
    return df


In [18]:
def get_episode_full(t,y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate_rate,
                     pool_init_x, pool_init_y, pool_init_wx, price_source, *args):
    
    """
    Purpuse: 
        generate one simulation based on selected parameters and random price change following the Browian motion.  
        pool attributes are rescorded at each step. 
        Rebate and fee values are back to the pool. 
    
    Input: 
        - t: term of the loan (e.g. 90 days)
        - y_price_init: initial price of loan token (e.g. btc = 50000 USD) 
        - bs_vol: black shoral option volatilty 
        - Growth_rate: linear growth slope of y price 
        - Real_vol: growth Brownie motion volatility 
        - Collateral amount (e.g. 200 BTC) 
        - LTV0: Initial LTV (e.g. 90%)
        - fee_rate: e.g. 0.15% of absolute weight changes 
        - rebate_rate: e.g. 50% impermanent loss due to rebalance weigths 
        - pool_init_x: Initial x size 
        - pool_init_y: Initial y size 
        - Pool_init_wx: Inital x weight (e.g. 50%)
        - price_source: [numpy array] actaul price or [0] generate price 
        
        Additional input: 
            - ('BS') Strickly follow BS
            - ('MA', 7) N: moving average window, padding 0.5 if t < N 
            - ('SM', 0.95) expotential smothing weight: 0.95 
            - ('50/50') reference 
    
    Output: 
        Dataframe with the following key measures. 
    
        - 1. Token price at time t 
        - 2. Reblanced weight at time t 
        - 3. Daily pool value loss (relative to collaterals)
                impermanet loss due to rebalance weights 
                impermanet loss due to price change 
        - 4. Collateral value 
        - 5. Collateral value with rebate 
        - 6. LTV relative to collateral with rebate
        - 7. Empirical impermanent loss 
        - 8. wt_change 
    """
    # call weights_generator class to simulate various weights rebalnce strategies  
    #print(args)
    wt_gen = weights_generator(t, y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol,price_source, 1- pool_init_wx)
    _out = wt_gen.BS_weights(args)
    w_btc = _out.weights_ytoken
    price = _out.y_price 
    #V0 = pool_init_y**w_btc[0]*pool_init_x**(1-w_btc[0]) 
    V0 = (Collateral*w_btc[0]/y_price_init)**w_btc[0]*(Collateral*(1-w_btc[0]))**(1-w_btc[0]) 

    borrowed_usd = Collateral*LTV0
    imp_loss_rebalance = _out.imp_loss_rebalance
    # calculate pool values 
    episode = gen_data_vec(w_btc, price, pool_init_x, pool_init_y, imp_loss_rebalance, V0, Collateral,rebate_rate)
    episode_full = pd.concat([_out, episode], axis=1)
    episode_full['ltv_with_rebate'] = borrowed_usd/episode_full['coll_with_rebate']    
    wt_values = episode_full['weights_ytoken'].values
    episode_full['wt_chg'] = wt_values - np.append(0.5, wt_values[:-1])
    episode_full['fee'] = np.abs(episode_full['wt_chg'].values)*np.append(Collateral, episode_full['coll_with_rebate'].values[:-1])*fee_rate
    episode_full['pvtc_rebate'] = episode_full['coll_with_rebate']/episode_full['y_price']/(Collateral/y_price_init)
    episode_full['value_loss_rebalance'] = np.append(Collateral,episode_full['coll_with_rebate'].values[:-1])*episode_full['imp_loss_rebalance'].values
    episode_full['value_loss_price'] = (episode_full['coll_with_rebate'].values - episode_full['value_loss_rebalance'].values)*episode_full['imp_loss_price'].values
    episode_full['value_loss_combined'] = episode_full['coll_with_rebate'].values - np.append(Collateral,episode_full['coll_with_rebate'].values[:-1] )
    return episode_full


In [ ]:
def episode_plot(var_list, *args, set_random_seed = True, row=2, col=3, Real_vol = 0.5, Growth_rate = 0, LTV0 = 0.8, bs_vol = 0.5,
    y_price_init = 50000, y_price_strike=50000, Collateral = 10000000, pool_init_x = 5000000, pool_init_y = 100, pool_init_wx = 0.5,
    fee_rate = 0,  rebate=0, price_source=[0]):
    %matplotlib notebook
    import matplotlib.pyplot as plt 
    fig, axs = plt.subplots(row,col,figsize=(8,6))
    axs[-1, -1].axis('off')
    _args = list(np.array(args).flat)
    # Create axes for sliders
    ax_growth = fig.add_axes([0.73, 0.26, 0.2, 0.02])
    ax_growth.spines['top'].set_visible(True)
    ax_growth.spines['right'].set_visible(True)

    ax_real_vol = fig.add_axes([0.73, 0.22, 0.2, 0.02])
    ax_real_vol.spines['top'].set_visible(True)
    ax_real_vol.spines['right'].set_visible(True)

    ax_ltv = fig.add_axes([0.73, 0.18, 0.2, 0.02])
    ax_ltv.spines['top'].set_visible(True)

    ax_bs_vol = fig.add_axes([0.73, 0.14, 0.2, 0.02])
    ax_bs_vol.spines['top'].set_visible(True)
    ax_bs_vol.spines['right'].set_visible(True)

    ax_rebate = fig.add_axes([0.73, 0.10, 0.2, 0.02])
    ax_rebate.spines['top'].set_visible(True)
    ax_rebate.spines['right'].set_visible(True)
    
    # Create sliders
    s_real_vol= Slider(ax=ax_real_vol, label='Real Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_growth = Slider(ax=ax_growth, label='Growth', valmin=-2, valmax=2, valinit=0, valfmt=' %1.2f', facecolor='#cc7000')
    s_bs_vol= Slider(ax=ax_bs_vol, label='BS Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_ltv = Slider(ax=ax_ltv, label='LTV0', valmin=0.7, valmax=0.95, valinit=0.9, valfmt=' %1.2f', facecolor='#cc7000')
    s_rebate = Slider(ax=ax_rebate, label='Rebate', valmin=0.0, valmax=1, valinit=0.5, valfmt=' %1.2f', facecolor='#cc7000')
    
    # Plot default data
    #t =  np.linspace(91,0,92)/365
    _data = get_episode_full(t,y_price_init, y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                     pool_init_x, pool_init_y, pool_init_wx,price_source, _args)
    x = _data.index.values
    y1 = _data[var_list[0]]/y_price_init
    y2a = _data[var_list[1]]
    y2b = 1-_data[var_list[1]]    
    y3 = _data[var_list[2]]
    y4 = _data[var_list[3]]
    y5a = _data[var_list[4]]
    y5b = _data[var_list[5]]
    y5c = _data[var_list[6]]
   
    f_d1, = axs[0,0].plot(x, y1, linewidth=2.5)
    f_d2a, = axs[0,1].plot(x, y2a, linewidth=2.5)
    f_d2b, = axs[0,1].plot(x, y2b, linewidth=2.5)
    f_d3, = axs[0,2].plot(x, y3, linewidth=2.5)
    f_d4, = axs[1,0].plot(x, y4, linewidth=2.5)
    f_d5a, = axs[1,1].plot(x, y5a, linewidth=1, linestyle = '--', color ='red', label= 'IL-_price')
    f_d5b, = axs[1,1].plot(x, y5b, linewidth=1, linestyle = '--', color ='green', label = 'IL_rebalance')
    f_d5c, = axs[1,1].plot(x, y5c, linewidth=1, color = 'black', linestyle='-', label='IL_empirical')
    

    axs[0,0].set_ylim(0.5,2.5)
    axs[0,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[0,1].set_ylim(-0.02,1.02)
    axs[0,1].axhline(y = 0.5, color = 'r', linestyle = '--')
    axs[0,2].set_ylim(0.5,2)
    axs[0,2].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,0].set_ylim(0.5,2)
    axs[1,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,1].set_ylim(-0.005,0)
        
    axs[0,0].set_ylabel('BTC relative change ')
    axs[0,1].set_ylabel('Weights (USDC: Orange, BTC: Blue)')
    axs[0,2].set_ylabel('LTV-with rebate')
    axs[1,0].set_ylabel('PVTC with rebate') 
    axs[1,1].set_ylabel('Impermanent Loss - log scale') 
    #axs[1,1].yscale('symlog')
    axs[1,1].legend()
    
#    axs[1,1].set_yticks(np.arange(-1, -8, step=-2), ['-10^1', '-10^3', '-10^5', '-10^7'])  # Set label locations.
    for i in range(2):
        for j in range(3):
            axs[i,j].set_xlabel('Day')
            axs[i,j].set_title('({})'.format(3*i+j+1))
    axs[1,1].set_xlabel('weight changes')
    # Update values
    def update(val):
        _real_vol = s_real_vol.val
        _growth = s_growth.val
        _bs_vol = s_bs_vol.val
        _ltv = s_ltv.val
        _rebate = s_rebate.val
        _data = get_episode_full(t,y_price_init,y_price_strike, _bs_vol, _growth, _real_vol, Collateral, _ltv,fee_rate, _rebate,
                     pool_init_x, pool_init_y, pool_init_wx, price_source,_args)
   
        x = _data.index.values
        _y1 = _data[var_list[0]]/y_price_init
        _y2a = _data[var_list[1]]
        _y2b = 1-_data[var_list[1]]    
        _y3 = _data[var_list[2]]
        _y4 = _data[var_list[3]]
        _y5a = _data[var_list[4]]
        _y5b = _data[var_list[5]]
        _y5c = _data[var_list[6]]

        f_d1.set_data(x,_y1)
        f_d2a.set_data(x,_y2a)
        f_d2b.set_data(x,_y2b)
        f_d3.set_data(x,_y3)
        f_d4.set_data(x,_y4)
        f_d5a.set_data(x,_y5a)
        f_d5b.set_data(x,_y5b)
        f_d5c.set_data(x,_y5c)
        fig.canvas.draw_idle()

    s_real_vol.on_changed(update)
    s_growth.on_changed(update)
    s_ltv.on_changed(update)
    s_bs_vol.on_changed(update)
    s_rebate.on_changed(update)
    fig.tight_layout()
    plt.show()

In [20]:
# if __name__ == '__main__':
    
#     set_random_seed = False
#     np.random.seed(101)
#     t =  np.linspace(91,0,92)/365
#     Real_vol =0.75
#     Growth_rate = -2
#     LTV0 = 0.8
#     bs_vol = 0.75
#     y_price_init = 50000
#     y_price_strike = 40000
#     Collateral = 10000000
#     pool_init_x = 2500000 
#     pool_init_y = 150
#     pool_init_wx = 0.25
#     fee_rate = 0.0
#     rebate=0
#     price_source = [0]
#     # special evaluations 
#     for _ in range(1):
#         example = get_episode_full(t,y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
#                               pool_init_x, pool_init_y, pool_init_wx,price_source,'SM', 0.95)

#         _x = np.diff(np.append(pool_init_x, example.x_locked.values))
#         _y = np.diff(np.append(pool_init_y, example.y_locked.values))
#         price = example.y_price.values
#         #_c0=_x + _y*price
#         #example['imp_empirical1'] = (_x + _y*price)/example.coll_with_rebate.values
#         #example['value_loss_combined'] = example.coll_with_rebate.values - np.append(Collateral,example.coll_with_rebate.values[:-1] )
#         # np.append(0,(np.diff(example.x_locked.values) + np.diff(example.y_locked.values)*example.y_price.values[1:])/example.coll_with_rebate.values[1:])

# #         # check on impermanent loss at various steps 
#         wt = example.weights_ytoken
#         rbpool = PoolEngine(x = pool_init_x, y = pool_init_y, w_x = pool_init_wx)
#         _check = []
#         for i in wt.index: 
#             if i == 0: 
#                 _check.append(np.append([i,int(1)], rbpool.step(0.5, price[i])))
#                 _check.append(np.append([i,int(2)], rbpool.step(1-wt[i], price[i])))
#             else: 
#                  _check.append(np.append([i,(1)], rbpool.step(1-wt[i-1], price[i])))
#                  _check.append(np.append([i,(2)], rbpool.step(1-wt[i], price[i])))
#         imp_loss = pd.DataFrame(_check, columns=['index', 'step', 'delta_x', 'delta_y', 'imp_loss', 'imp_value_loss',\
#                                                 'value_pnl_pct', 'value_pnl'])
#         imp_loss =imp_loss.set_index(['index','step']).unstack().add_prefix('').reset_index()
#         imp_loss.columns=imp_loss.columns.map('_'.join)
#         check= pd.concat([imp_loss, example[['imp_loss_price', 'imp_loss_rebalance','imp_empirical', 'value_loss_combined']]], axis=1)
# #        check.to_csv('imp_loss.csv', index=False)
# #         check

In [21]:
def ltv_simulation(s, *args ):
    """
        MC simualtion
        output: dataframe of the following columns
         ['pool_value', 
          'pvtc', 
          'default':  at anytime 
          'portfolio', 
          'imp_weight', 
          'imp_price', 
          'PNL_price',
          'total_pnl',
          'mean_abs_wt_change',
          'loss_at_default', 
          'wt_default', 
          'wt_final'
          'default_at_maturity'
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init, y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source, args)
        #cur_loss = 1-(1+_out.imp_empirical.values).cumprod()
        _imp_rebalance = _out.value_loss_rebalance.sum()
        _imp_price  = _out.value_loss_price.sum()
        # PnL due to price change 
        _PNL = _out.coll_with_rebate.values[-1] - Collateral -_imp_rebalance-_imp_price
        _total_pnl = _out.coll_with_rebate.values[-1] - Collateral
        # default at any time 
        default = _out.ltv_with_rebate.values.max()>1
        default_at_maturity = _out.ltv_with_rebate.values[-1]>1
        _loss_index = np.argwhere(_out.ltv_with_rebate.values>1)
        if default >0: 
            _loss = _out.coll_with_rebate.values[_loss_index[0][0]] - Collateral*LTV0
            wt_default = _out.weights_ytoken.values[_loss_index[0][0]]
        else:
            _loss = 0 
            wt_default = _out.weights_ytoken.values[-1]
        wt_final = _out.weights_ytoken.values[-1]
        mean_wt_change = np.abs(_out.wt_chg.values).mean()
        pv_minus_bnh = _out.coll_with_rebate.values[-1] - 200*_out.y_price.values[-1]
        bnh_default = (Collateral*LTV0/(200*_out.y_price.values[-1]))>1
        pv_minus_bnh_btc = _out.coll_with_rebate.values[-1]/_out.y_price.values[-1] - 200
        out.append([_out.coll_with_rebate.values[-1], _out.pvtc_rebate.values[-1], 
                    default, 200*_out.y_price.values[-1], 
                    _imp_rebalance, _imp_price,_PNL, _total_pnl, mean_wt_change, _loss, wt_default,
                    wt_final,default_at_maturity, pv_minus_bnh, bnh_default,pv_minus_bnh_btc])
    return np.array(out)

9960189.35

,t,y_price,weights_ytoken,imp_loss_rebalance,imp_loss_price,delta_wt,x_locked,y_locked,V,coll_with_rebate,...,imp_empirical,delta_x,delta_y,ltv_with_rebate,wt_chg,fee,pvtc_rebate,value_loss_rebalance,value_loss_price,value_loss_combined
0,0.249315,50000.000000,0.751661,-0.000007,-2.007425e-06,7.832185e-01,2.474752e+06,150.505358,1.677044e+03,9.965215e+06,...,-9.682488e-06,-25247.857777,0.505358,0.802793,0.251661,0.0,0.996521,-73.673445,-20.004570,-34785.489647
1,0.246575,49768.742235,0.753089,-0.000005,-5.099047e-08,7.802184e-01,2.461882e+06,150.762865,1.653958e+03,9.970720e+06,...,-4.464308e-06,-12869.775011,0.257507,0.802349,0.001428,0.0,1.001705,-54.491143,-0.508414,5505.123965
2,0.243836,49805.615419,0.754508,-0.000005,-6.154998e-04,7.814738e-01,2.302634e+06,154.079133,1.631343e+03,9.379671e+06,...,-6.948803e-04,-159248.577647,3.316268,0.852908,0.001419,0.0,0.941628,-54.073786,-5773.218838,-591048.445879
3,0.241096,45931.186597,0.752389,-0.000012,-1.773054e-04,7.121183e-01,2.247751e+06,155.307241,1.664919e+03,9.077735e+06,...,-9.317932e-05,-54882.782484,1.228108,0.881277,-0.002119,0.0,0.988189,-113.520760,-1609.551902,-301936.242941
4,0.238356,43977.240946,0.748306,-0.000045,-1.513707e-05,6.707420e-01,2.306509e+06,153.965073,1.731322e+03,9.163951e+06,...,-1.124117e-04,58758.294217,-1.342168,0.872986,-0.004082,0.0,1.041897,-404.531195,-138.721449,86215.849736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0.010959,29085.908769,0.164873,-0.000266,-8.635506e-05,2.922706e-05,5.725059e+06,40.265744,8.095014e+05,6.855311e+06,...,-4.713308e-05,24647.716757,-0.889668,1.166978,-0.008676,0.0,1.178459,-1834.136449,-592.149215,-42142.465279
88,0.008219,28069.824486,0.156629,-0.000250,-3.323715e-04,1.139575e-07,5.715729e+06,40.620921,8.926798e+05,6.777241e+06,...,-7.063419e-06,-9329.945127,0.355177,1.180421,-0.008244,0.0,1.207211,-1714.760003,-2253.131635,-78070.642708
89,0.005479,26132.142159,0.148797,-0.000235,-1.293631e-04,1.000000e-08,5.806161e+06,37.132162,9.795226e+05,6.821127e+06,...,-7.273291e-04,90432.056065,-3.488759,1.172827,-0.007831,0.0,1.305122,-1595.054070,-882.608624,43886.705509
90,0.002740,27333.887201,0.141358,-0.000222,-2.252898e-04,1.000000e-08,5.805024e+06,37.176364,1.070676e+06,6.760700e+06,...,-1.154331e-07,-1137.062522,0.044202,1.183309,-0.007440,0.0,1.236688,-1511.367991,-1523.457466,-60427.220547


In [31]:
def ltv_simulation_stoploss(s, stop_limit, convert_slip,  *args ):
    """
        MC simualtion
        output: dataframe of the following columns
         ['pool_value', 
          'pvtc', 
          'default':  at anytime 
          'portfolio', 
          'imp_weight', 
          'imp_price', 
          'PNL_price',
          'total_pnl',
          'mean_abs_wt_change',
          'loss_at_default', 
          'wt_default', 
          'wt_final'
          'default_at_maturity'
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init, y_price_strike,bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source, args)
        # convert all to usd if stop loss  
        convert = 0
        insolvent = 0 
        insolvent_pct = 0 
        _pv = _out.coll_with_rebate.values
        x_value = _out.x_locked.values
        y_price = _out.y_price.values 
        y_price = np.append(y_price, y_price[-1])
        y_value = _out.y_locked.values * y_price[1:]* convert_slip
        _convert_index = np.argwhere(_pv <= stop_limit)
        if len(_convert_index)>0: 
            convert = 1
            final_pv = x_value[_convert_index[0][0]] + y_value[_convert_index[0][0]]
            wt_convert = _out.weights_ytoken.values[_convert_index[0][0]]
            wt_final = 1
            value_loss_rebalance = -1*_out.value_loss_rebalance[:_convert_index[0][0]].sum()/Collateral  # net loss amount  
        else:
            final_pv = _pv[-1] 
            wt_convert = _out.weights_ytoken.values[-1]
            wt_final = _out.weights_ytoken.values[-1]
            value_loss_rebalance = -1*_out.value_loss_rebalance.sum()/Collateral  # net loss amount  
        pv_to_collateral = final_pv/Collateral
        pv_to_bnh = final_pv/(pool_init_y/(1-pool_init_wx)*_out.y_price.values[-1])
        
        if final_pv < Collateral*LTV0: 
            insolvent = 1
            insolvent_pct = -1*(final_pv - Collateral*LTV0)/Collateral # net insolvent amount
        out.append([convert, insolvent, pv_to_collateral, pv_to_bnh, insolvent_pct,value_loss_rebalance, wt_convert])
    return np.array(out)

In [25]:
# import pandas as pd 
# if __name__ == '__main__':
#     #stop_limit = Collateral*ltv*1.07
#     s=1 
#     print(stop_limit)
#     _sm = pd.DataFrame(ltv_simulation_stoploss(s, stop_limit, 'SM', 0.95))

7490000.0

In [27]:
def ltv_corr(s, *args ):
    """
        MC simualtion
        output: dataframe of the following columns
         ['pnl_price_corr', 
          'pnl_price_corr_pos',
          'pnl_price_corr_neg']
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source, args)
        #cur_loss = 1-(1+_out.imp_empirical.values).cumprod()
        _total_pnl = _out.coll_with_rebate.values
        _total_pnl_diff = _total_pnl[1:] - _total_pnl[:-1]
        _price = _out.y_price.values
        _price_pad = np.append(_price, _price[-1])
        _price_diff = _price_pad[1:] - _price_pad[:-1]
        _price_diff = _price_diff[1:]
        idx = _price_diff>=0 
        out.append([np.corrcoef(_total_pnl_diff, _price_diff)[0][1],\
                    np.corrcoef(_total_pnl_diff[idx], _price_diff[idx])[0][1],\
                    np.corrcoef(_total_pnl_diff[~idx], _price_diff[~idx])[0][1]])
    return np.array(out)




array([1, 3, 3])

In [29]:
def ltv_simulation_path(s, *args ):
    """
        MC simualtion
            check the distribution of final pool value and risk of default 
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source, args)
        _out['sim_id'] = int(k)
        if k== 0: 
            out = _out
        else: 
            out = pd.concat([out, _out])
            
    return out


def ltv_simulation_path_stoploss(s, convert_slip, stop_limit, *args ):
    """
        MC simualtion
            check the distribution of final pool value and risk of default 
            add conversion threshold 
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init,y_price_strike, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source, args)
        _out['sim_id'] = int(k)
        convert = np.zeros(_out.shape[0])        
        _pv = _out.coll_with_rebate.values
        y_weigth = _out.weights_ytoken.values
        x_value = _out.x_locked.values
        y_price = _out.y_price.values 
        y_price = np.append(y_price, y_price[-1])
        y_value = _out.y_locked.values * y_price[1:]* convert_slip
        _convert_index = np.argwhere(_pv <= stop_limit)
        if len(_convert_index)>0: 
            _first= _convert_index[0][0] 
            convert[_first:] = 1
            x_value[_first:] = x_value[_first] + y_value[_first]
            y_value[_first:] = 0 
        _out['new_x_value'] = x_value
        _out['new_y_value'] = y_value
        _out['convert'] = convert
    return _out

In [30]:
import matplotlib.ticker as mtick
def plot_errbar(df, cols, label=''):
    import matplotlib.pyplot as plt
    import numpy as np
    
    _mean = df.groupby(cols[1]).mean().reset_index()
    phat = _mean[cols[0]].values
    _std = np.sqrt(phat*(1-phat))
    fig, ax = plt.subplots(figsize=(8,6))
    ax.errorbar(_mean[cols[1]], _mean[cols[0]], _std, linestyle='None', marker='^')
    ax.set_xlabel(cols[1])
    ax.set_ylabel(cols[0])
    ax.set_title('{} by {}'.format(cols[0], cols[1]))
    plt.xticks(rotation = 45)
    plt.savefig('{}_by_{}_{}.png'.format(cols[0], cols[1], label))
    plt.show()

def plot_boxplt(df, cols, label=''):
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(8,6))
    sns.boxplot(x = cols[1], y = cols[0], data = df)
    ax.set_xlabel(cols[1])
    ax.set_ylabel(cols[0])
    #ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
    plt.xticks(rotation = 45)
    plt.yscale('symlog')
    ax.set_title('{} by {} - {}'.format(cols[0], cols[1],label))
    plt.savefig('{}_by_{}_{}.png'.format(cols[0], cols[1],label))
    plt.show()
    
def sim_summary(df, cols, plot, label=''):
    _df = df[cols]
    #MA.groupby('ma_window').describe().unstack(1)
    A = _df.groupby(cols[1])[cols[0]].describe()
    b= _df.groupby(cols[1])[cols[0]].skew()
    b.name = 'skew'
    c = _df.groupby(cols[1])[cols[0]].apply(pd.DataFrame.kurt)
    c.name = 'kurt'
    if plot==1: 
        plot_boxplt(df, cols, label)
    elif plot==2:
        plot_errbar(df, cols, label)
    return pd.concat([A,b,c], axis=1)





In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt


def _3d_plot(df, x_var, y_var, z_var):
    _df1 = df
    xlist = _df1[x_var].unique()
    _xlist = xlist.astype(np.float)
    ylist = _df1[y_var].unique()
    _ylist = ylist.astype(np.float)
    X,Y = np.meshgrid(_xlist,_ylist)
    zp = np.ndarray((_xlist.shape[0],_ylist.shape[0]))

    for x in range(0, len(_xlist)):
        for y in range(0, len(_ylist)):
            zp[x][y] = _df1.loc[(_df1[x_var] == xlist[x]) & (_df1[y_var]==ylist[y])][z_var]
    Z = zp.T
    fig = plt.figure(figsize=(10,6))
    ax = Axes3D(fig)
    surf = ax.plot_trisurf(X.flatten(),Y.flatten(),Z.flatten(), cmap=cm.jet, linewidth=0.1)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.set_title('{} Surfact Plot '.format(z_var))
    ax.set_xlabel(x_var)
    ax.set_ylabel(y_var)
    return fig


def contour_plot(df, x_var, y_var, z_var):
    import matplotlib.backends.backend_pdf
    from matplotlib import cm
    _df1 = df
    xlist = _df1[x_var].unique()
    _xlist = xlist.astype(np.float)
    ylist = _df1[y_var].unique()
    _ylist = ylist.astype(np.float)

    X,Y = np.meshgrid(_xlist,_ylist)
    zp = np.ndarray((_xlist.shape[0],_ylist.shape[0]))

    for x in range(0, len(_xlist)):
        for y in range(0, len(_ylist)):
            zp[x][y] = _df1.loc[(_df1[x_var] == xlist[x]) & (_df1[y_var]==ylist[y])][z_var]
    Z = zp.T
    fig, axs = plt.subplots(1,1,figsize=(10,6))
    sp = axs.contourf(X, Y, Z,  cmap = 'jet', vmin=0,vmax=1)
    axs.set_title('{} Contour Plot '.format(z_var))
    axs.set_xlabel(x_var)
    axs.set_ylabel(y_var)
    fig.colorbar(sp)
    return fig 
